# TNEG Geospatial Visualizations

This notebook contains a base map with county-level breakdown for the state of TN.  

The following dataframes have been joined to the map:  
* ``earthquakes`` Deadly earthquakes since 1900 (from Wikipedia)
* ``usgs`` Earthquakes in and around TN since 1900 (from the USGS API)
* ``TN_demo`` TN county-level demographic information including total population, % of children (under 18), % of people living in rural or isolated settings, % of people of color, % of people with disabilities, and % of senior citizens (from the TN Arts Commission, derived from the 2010 US census)
* ``TN_housing_units_by_county`` TN county-level aggregates of total population and number of housing units (from the 2010 census)
* ``acs_data``

In [ ]:
# import statements
import pandas as pd
import numpy as np
import requests
import ipywidgets as widgets
#from bs4 import BeautifulSoup as bs
from IPython.core.display import HTML
#import matplotlib.pyplot as plt
import matplotlib as mpl
import pylab as plt
import json
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.models.widgets import DataTable
from bokeh.plotting import figure
from bokeh.palettes import brewer
import panel as pn
import panel.widgets as pnw
#import seaborn as sns
from io import StringIO
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from cartopy.io import shapereader
%matplotlib inline
# import io
# import scipy.stats as stats
# import statsmodels.api as sm

In [ ]:
# display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Create a base map for the state of TN, broken down by county.

In [ ]:
# Create a folium map, but probably not the best idea to subsequently join all of the other data
# tn_map = folium.Map(location=[36,-86], zoom_start = 7)
# tn_map

In [ ]:
# Create a base map by importing the base shape file for TN census map
# Annoyingly, this is broken down by census division, not by county
# tn_census_map = gpd.read_file('../data/2018-TN-basemap/tl_2016_47_cousub.shp')
# tn_census_map.info()

In [ ]:
# Check to see if COUNTYFP |is the right level to aggregate at to get county-level geometries
# tn_census_map['COUNTYFP'].nunique()

In [ ]:
# Attempt to aggregate geometries by county - and not succeed
# tn_census_map.groupby('COUNTYFP')['geometry']

In [ ]:
# Ask Michael, get this county-level base map shapefile instead
tn_county_map = gpd.read_file('../data/TN-county-basemap/tncounty.shp')
# See what the map looks like
tn_county_map.plot();

In [ ]:
# Take a look at the base map dataframe
tn_county_map.head()

In [ ]:
# Check to make sure nothing is missing
tn_county_map.info()

In [ ]:
# check the projection type
tn_county_map.crs

In [ ]:
# change the projection type
tn_county_map = tn_county_map.to_crs('EPSG:4326')
print(tn_county_map.crs)
# Clean up some of the columns we don't need
tn_county_map = tn_county_map.drop(['OBJECTID', 'KEY', 'SHAPE_LEN'], axis = 1)
# Reformat column headers
tn_county_map.columns = ['county', 'shape_area', 'geometry']
# Set the county names to lower case
tn_county_map['county'] = tn_county_map['county'].str.lower()
# Make sure the base map dataset is good to go
tn_county_map.head(2)

### Read in the deadly earthquakes since 1900 wikitable and turn it into a Geo DataFrame.

In [ ]:
earthquakes = pd.read_csv('../data/earthquakes_wikitable.csv')
earthquakes.head()

In [ ]:
# Check the data types
earthquakes.info()

In [ ]:
# Set the origin_utc column to a datetime
earthquakes['origin_utc'] = pd.to_datetime(earthquakes['origin_utc'])
# Create 
earthquakes['magnitude_type'] = earthquakes['magnitude'].str.extract(r'.*?(\w+)$')
# Clean up the lat/long columns
earthquakes['latitude'] = earthquakes['lat'].str.replace('?','')
earthquakes['longitude'] = earthquakes['long'].str.replace('?','')
# Turn the lat/long into floats
earthquakes['latitude'] = pd.to_numeric(earthquakes['latitude'], errors = 'raise')
earthquakes['longitude'] = pd.to_numeric(earthquakes['longitude'], errors = 'raise')
# Remove excess columns
earthquakes = earthquakes.drop(['date_ymd', 'time', 'lat', 'long', 'magnitude', 'pde_shaking_deaths', 'pde_total_deaths', 'utsu_total_deaths', 'em_dat_total_deaths', 'other_source_deaths', 'other_source_deaths_new', 'osd1', 'osd2', 'osd3'], axis = 1)
# Reorder columns
earthquakes = earthquakes[['origin_utc', 'country', 'latitude', 'longitude', 'depth_km', 'magnitude_num', 'magnitude_type', 'secondary_effects', 'max_deaths']]
# Make sure the data types are correct
earthquakes.info()

In [ ]:
# Create a new column named 'geometry' which combines the latitude and longitude
earthquakes['geometry'] = earthquakes.apply(lambda x: Point((float(x.longitude),
                                                            float(x.latitude))),
                                           axis = 1)
earthquakes.head()

In [ ]:
# Turn the Wikipedia Deadly Earthquakes since 1900 table into a Geo Data Frame
earthquakes_geo = gpd.GeoDataFrame(earthquakes,
                                   crs = tn_county_map.crs,
                                   geometry = earthquakes['geometry'])

The Wikipedia deadly earthquakes since 1900 table is now ready for a spatial join.

### Next, pull in the USGS data for earthquakes in TN and turn it into a Geo DataFrame.  

Use the coordinates in this [gist](https://gist.github.com/jakebathman/719e8416191ba14bb6e700fc2d5fccc5) to only get earthquakes from near TN.

In [ ]:
# Set the query URL
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-10-22&minlatitude=34.9884&maxlatitude=36.6871&minlongitude=-90.3131&maxlongitude=-81.6518'
# Assign the response to a variable
r = requests.get(url)

In [ ]:
# Read in the text of the response into a dataframe called usgs
usgs = pd.read_csv(StringIO(r.text))
# See what gets returned
usgs.head()

In [ ]:
# Make sure all of the fields are the proper data types
usgs.info()

In [ ]:
# Remove excess columns
usgs = usgs.drop(['nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'status', 'locationSource', 'magSource'], axis = 1)
usgs.info()

In [ ]:
# Create a geometry column
usgs['geometry'] = usgs.apply(lambda x: Point((float(x.longitude),
                                            float(x.latitude))),
                              axis = 1)
# Make sure it worked
usgs.head()

In [ ]:
# Subset to earthquakes that have happened within TN only, based on the place name
# Not actually necessary if we're going to plot points based on lat/long
# usgs_tn = usgs[usgs['place'].str.contains('(Tennessee)')]
# usgs_tn.info()

In [ ]:
# Turn the USGS dataframe into a Geo DataFrame
usgs_geo = gpd.GeoDataFrame(usgs,
                            crs = tn_county_map.crs,
                            geometry = usgs['geometry'])

The usgs dataframe is now ready for a spatial join.

### Read in the TN demographics data by county

In [ ]:
tn_demo = pd.read_csv('../data/TN-county-demographics-2010.csv')
# Check the top of the tn_demo dataframe
tn_demo.head()

In [ ]:
# lower case the county to avoid merge errors
tn_demo['County'] = tn_demo['County'].str.lower()
# Rename columns
tn_demo.columns = ['county', 'total_pop', 'pct_children_under_18', 'pct_people_living_in_rural_areas', 'pct_people_of_color', 'pct_people_with_disabilities', 'pct_senior_citizens']
tn_demo.info()

In [ ]:
# remove non-numeric characters from the columns
tn_demo[['pct_children_under_18', 'pct_people_living_in_rural_areas', 'pct_people_of_color', 'pct_people_with_disabilities', 'pct_senior_citizens']] = tn_demo[['pct_children_under_18', 'pct_people_living_in_rural_areas', 'pct_people_of_color', 'pct_people_with_disabilities', 'pct_senior_citizens']].apply(lambda x: x.str.replace('%',''))
tn_demo.total_pop = tn_demo.total_pop.str.replace(',','')
tn_demo

In [ ]:
tn_demo[['total_pop', 'pct_children_under_18', 'pct_people_living_in_rural_areas', 'pct_people_of_color', 'pct_people_with_disabilities', 'pct_senior_citizens']] = tn_demo[['total_pop', 'pct_children_under_18', 'pct_people_living_in_rural_areas', 'pct_people_of_color', 'pct_people_with_disabilities', 'pct_senior_citizens']].apply(pd.to_numeric, errors = 'raise')
tn_demo.info()

The TN demographics dataframe is now ready for a non-spatial join.

### Read in and clean up the census housing units by county for TN data.

In [ ]:
# Only read in the relevant lines of the csv
tn_housing = pd.read_csv('../data/us_census_tn_housing_units_by_county_2010-2019.csv')[2:99]

In [ ]:
# Use the first row as the column headers
tn_housing.columns = tn_housing.iloc[0]
# Remove the excess lines from the dataframe
tn_housing = tn_housing[2:99]

In [ ]:
# Reset the index
tn_housing = tn_housing.reset_index(drop = True)
tn_housing

In [ ]:
# Drop the excess columns
tn_housing = tn_housing.drop(['Census', 'Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis = 1)
# Rename the columns
tn_housing.columns = ['county', 'total_housing_units_2019']
# Clean up and standardize the county names
tn_housing['county'] = tn_housing['county'].str.extract(r'\.(.*) County, Tennessee')
tn_housing['county'] = tn_housing['county'].str.lower()
tn_housing['total_housing_units_2019'] = tn_housing['total_housing_units_2019'].str.replace(',','')
tn_housing['total_housing_units_2019'] = pd.to_numeric(tn_housing['total_housing_units_2019'], errors = 'raise')

In [ ]:
tn_housing.info()

The TN housing units dataset is now ready to join.

### Read in the ACS dataframe

In [ ]:
acs = pd.read_csv('../data/acs_5yr_subset_clean.csv')
acs.head()

In [ ]:
acs.info()

The ACS data has already been cleaned and is ready for a non-spatial join.

### Read in the CDC Social Vulnerability Index dataframe

From [here](https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html)

In [ ]:
# Only read in the Social Vulnerability Index scores for the counties in TN
svi = pd.read_csv('../data/TN-vulnerability-scores.csv', usecols=['COUNTY','AREA_SQMI','RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES'])
svi.head()

In [ ]:
svi.info()

In [ ]:
# rename the columns
svi.columns = ['county', 'area_m2', 'socioeconomic', 'household_comp_and_disability', 'minority_status_and_language', 'housing_type_and_transportation', 'total_vulnerability']
svi.info()

In [ ]:
# convert the county to lowercase to facilitate joins
svi['county'] = svi['county'].str.lower()

In [ ]:
# sort the dataset by the county, then reset the index
svi = svi.sort_values('county').reset_index(drop=True)
svi

The svi dataset is ready to join.

### Merge the non-spatial dataframes together  

Merge on the county level to build the framework for mapping the data.

In [ ]:
# Join the demographics and housing data into one dataframe
tn_housing_demo = tn_demo.merge(tn_housing, how = 'outer', on = 'county')
tn_housing_demo.info()

In [ ]:
tn_housing_demo_acs = tn_housing_demo.merge(acs, how = 'outer', on = 'county')
tn_housing_demo_acs.info()

In [ ]:
tn_housing_demo_acs_svi = tn_housing_demo_acs.merge(svi, how = 'outer', on = 'county')
tn_housing_demo_acs_svi.info()

In [ ]:
# Rename the final dataframe for ease of use
tn_data = tn_housing_demo_acs_svi

### Merge the non-spatial aggregate dataframe to the base map

In [ ]:
# Join in the demographic data to the TN base map
tn_data_map = tn_county_map.merge(tn_data, on = 'county')
# Check to make sure it is a full join
tn_data_map

In [ ]:
# Check to make sure nothing got dropped along the way
tn_data_map.info()

In [ ]:
# Start exploring the data visually
ax = tn_data_map.plot(column='total_vulnerability', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 legend = True)

In [ ]:
# Build a widget
@widgets.interact(
    column = ['total_pop', 'pct_children_under_18', 'pct_people_living_in_rural_areas'])

def throw_some_shade():
    """
    Change the choropleth based on the column value
    """
    ax = tn_data_map.plot(column='total_pop', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 legend = True)
    ax.plot(column = 'total_pop')
#widgets.interact(throw_some_shade, col=['total_pop', 'pct_children_under_18', 'pct_people_living_in_rural_areas']);

### Add of the USGS earthquakes since 1900 to the data-enriched TN base map

In [ ]:
# Join the USGS data to the TN basemap
tn_earthquakes = gpd.sjoin(usgs_geo, tn_data_map, op = 'within')
tn_earthquakes

In [ ]:
# Since 1900, how many earthquakes per county?
tn_earthquakes['county'].value_counts()

In [ ]:
# map the earthquakes, color-coded by magnitute
ax = tn_county_map.plot(figsize = (8, 10), color = 'beige')
tn_earthquakes.plot( ax = ax, column = 'mag');
plt.show();

In [ ]:
# map the earthquakes, color-coded by county
ax = tn_county_map.plot(figsize = (8, 10), color = 'beige')
tn_earthquakes.plot( ax = ax, column = 'county');
plt.show();

In [ ]:
tn_earthquakes['county'].value_counts().hist(bins = 35);

In [ ]:
tn_earthquakes['mag'].hist(bins = 35);

In [ ]:
tn_earthquakes['depth'].hist(bins = 35);